In [1]:
!pip install "pymilvus[model]==2.5.10" openai==1.82.0 requests==2.32.3 tqdm==4.67.1 torch==2.7.0

Looking in indexes: http://mirrors.cloud.aliyuncs.com/pypi/simple/


In [83]:
import os

api_key = os.getenv("DEEPSEEK_API_KEY")

In [64]:
from glob import glob

text_lines = []

for file_path in glob("./mfd.md", recursive=True):
    # 以只读模式读取文件内容
    with open(file_path, "r") as file:
        file_text = file.read()

    # 以空行分割段落
    text_lines += file_text.split("# ")

In [65]:
len(text_lines)

30

In [66]:
# 定义向量模型
from pymilvus import model as milvus_model

embedding_model = milvus_model.DefaultEmbeddingFunction()

# 获取文本向量维度
test_embedding = embedding_model.encode_queries(["测试文本"])[0]
embedding_dim = len(test_embedding)
print(embedding_dim)

768


In [69]:
# 创建 collection
from pymilvus import MilvusClient

milvus_client = MilvusClient(uri="./my_milvus.db")
collection_name = "mfd_collection"

In [70]:
if milvus_client.has_collection(collection_name):
    milvus_client.drop_collection(collection_name)

In [71]:
milvus_client.create_collection(
    collection_name=collection_name,
    dimension=embedding_dim,
    metric_type="IP",
    consistency_level="Strong"
)

In [72]:
# 数据插入
# 可视化进度
from tqdm import tqdm

data = []

doc_embeddings = embedding_model.encode_documents(text_lines)

for i, line in enumerate(tqdm(text_lines, desc="creating embeddings")):
    data.append({ "id": i, "vector": doc_embeddings[i], "text": line })

milvus_client.insert(collection_name=collection_name, data=data)

creating embeddings: 100%|██████████| 30/30 [00:00<00:00, 285975.27it/s]


{'insert_count': 30, 'ids': [0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29], 'cost': 0}

In [89]:
# 构建问题
question = "合同的变更和转让"

In [90]:
# 在 milvus collection 中获取相关的法律条例
search_res = milvus_client.search(
    collection_name=collection_name,
    data=embedding_model.encode_queries([question]),
    limit=5,
    search_params={ "metric_type": "IP", "params": {} },
    output_fields=["text"]
)

In [91]:
import json

retrieved_lines_with_distances = [
    (res["entity"]["text"], res["distance"]) for res in search_res[0]
]

print(json.dumps(retrieved_lines_with_distances, indent=4, ensure_ascii=False))

[
    [
        "第三章 合同的变更和转让\n\n**第五百四十八条** 当事人协商一致，可以变更合同。\n\n**第五百四十九条** 当事人对合同变更的内容约定不明确的，推定为未变更。\n\n**第五百五十条** 债权人可以将合同的权利全部或者部分转让给第三人，但是有下列情形之一的除外：\n（一）根据合同性质不得转让；\n（二）按照当事人约定不得转让；\n（三）依照法律规定不得转让。\n债权人转让权利的，应当通知债务人。未经通知，该转让对债务人不发生效力。\n\n**第五百五十一条** 债权人转让权利的，受让人取得与债权有关的从权利，但是该从权利专属于债权人自身的除外。\n\n**第五百五十二条** 债务人将合同的义务全部或者部分转让给第三人的，应当经债权人同意。\n\n**第五百五十三条** 债务人转让义务的，新债务人可以主张原债务人对债权人的抗辩。\n新债务人承担债务的，应当承担与主债务有关的从债务，但是该从债务专属于原债务人自身的除外。\n\n**第五百五十四条** 当事人一方经对方同意，可以将自己在合同中的权利和义务一并转让给第三人。\n\n**第五百五十五条** 权利和义务一并转让的，适用债权转让、债务转让的有关规定。\n\n**第五百五十六条** 合同变更的，不影响当事人请求损害赔偿的权利。\n\n###",
        0.6202714443206787
    ],
    [
        "第五章 占有\n\n**第四百七十一条** 占有是指对物事实上的控制和支配。\n\n**第四百七十二条** 占有可以分为直接占有和间接占有。\n直接占有是指直接对物进行控制和支配。\n间接占有是指通过他人对物进行控制和支配。\n\n**第四百七十三条** 占有的取得和消灭，适用本法有关物权设立和消灭的规定。\n\n**第四百七十四条** 占有人合法占有动产的，善意取得人取得该动产所有权。\n占有人非法占有动产的，善意取得人取得该动产所有权。\n\n**第四百七十五条** 占有物毁损、灭失的，占有人应当承担赔偿责任。\n占有人善意占有物的，不承担赔偿责任。\n占有人恶意占有物的，应当承担赔偿责任。\n\n**第四百七十六条** 占有被侵夺的，占有人有权请求返还原物。\n占有物毁损、灭失的，占有人有权请求赔偿损失。\n\n**第四百七十七

In [92]:
context = "\n".join(
    (line_with_distance[0]) for line_with_distance in retrieved_lines_with_distances
)

In [93]:
context

'第三章 合同的变更和转让\n\n**第五百四十八条** 当事人协商一致，可以变更合同。\n\n**第五百四十九条** 当事人对合同变更的内容约定不明确的，推定为未变更。\n\n**第五百五十条** 债权人可以将合同的权利全部或者部分转让给第三人，但是有下列情形之一的除外：\n（一）根据合同性质不得转让；\n（二）按照当事人约定不得转让；\n（三）依照法律规定不得转让。\n债权人转让权利的，应当通知债务人。未经通知，该转让对债务人不发生效力。\n\n**第五百五十一条** 债权人转让权利的，受让人取得与债权有关的从权利，但是该从权利专属于债权人自身的除外。\n\n**第五百五十二条** 债务人将合同的义务全部或者部分转让给第三人的，应当经债权人同意。\n\n**第五百五十三条** 债务人转让义务的，新债务人可以主张原债务人对债权人的抗辩。\n新债务人承担债务的，应当承担与主债务有关的从债务，但是该从债务专属于原债务人自身的除外。\n\n**第五百五十四条** 当事人一方经对方同意，可以将自己在合同中的权利和义务一并转让给第三人。\n\n**第五百五十五条** 权利和义务一并转让的，适用债权转让、债务转让的有关规定。\n\n**第五百五十六条** 合同变更的，不影响当事人请求损害赔偿的权利。\n\n###\n第五章 占有\n\n**第四百七十一条** 占有是指对物事实上的控制和支配。\n\n**第四百七十二条** 占有可以分为直接占有和间接占有。\n直接占有是指直接对物进行控制和支配。\n间接占有是指通过他人对物进行控制和支配。\n\n**第四百七十三条** 占有的取得和消灭，适用本法有关物权设立和消灭的规定。\n\n**第四百七十四条** 占有人合法占有动产的，善意取得人取得该动产所有权。\n占有人非法占有动产的，善意取得人取得该动产所有权。\n\n**第四百七十五条** 占有物毁损、灭失的，占有人应当承担赔偿责任。\n占有人善意占有物的，不承担赔偿责任。\n占有人恶意占有物的，应当承担赔偿责任。\n\n**第四百七十六条** 占有被侵夺的，占有人有权请求返还原物。\n占有物毁损、灭失的，占有人有权请求赔偿损失。\n\n**第四百七十七条** 占有被侵夺的，占有人有权请求返还原物。\n占有物毁损、灭失的，占有人有权请求赔偿损失。\n\n**第四百七十八条** 占有物

In [94]:
question

'合同的变更和转让'

In [95]:
# 定义 deepseek 问答助手
from openai import OpenAI

deepseek_client = OpenAI(
    api_key=api_key,
    base_url = "https://api.deepseek.com/v1"
)

In [100]:
# 定义 prompt
SYSTEM_PROMPT = """
你是一个 ai 问答小助手，你可以使用提供的上下文回答问题
"""
USER_PROMPT = f"""
请使用<context>标签内的上下文来回答<question>标签内的问题
<question>
{question}
</question>
<context>
{context}
</context>
"""

In [101]:
response=deepseek_client.chat.completions.create(
    model="deepseek-chat",
    messages=[
        {"role": "system", "content": SYSTEM_PROMPT},
        {"role": "user", "content": USER_PROMPT}
    ]
)
print(response.choices[0].message.content)

根据《context》中第三章关于"合同的变更和转让"的规定，主要内容如下：

一、合同变更
1. 变更条件（第五百四十八条）：需当事人协商一致方可变更合同。
2. 变更不明效力（第五百四十九条）：对变更内容约定不明确的，推定为未变更。
3. 变更后果（第五百五十六条）：合同变更不影响当事人请求损害赔偿的权利。

二、合同权利转让（债权转让）
1. 转让原则（第五百五十条）：
   - 允许全部或部分转让给第三人
   - 三种禁止情形：依合同性质/当事人约定/法律规定不得转让
   - 需通知债务人，否则对债务人不生效
2. 从权利处理（第五百五十一条）：受让人取得与债权相关的从权利（专属于债权人的除外）

三、合同义务转让（债务承担）
1. 转让条件（第五百五十二条）：须经债权人同意
2. 抗辩权继承（第五百五十三条）：
   - 新债务人可主张原债务人的抗辩
   - 需承担与主债务相关的从债务（专属于原债务人的除外）

四、权利义务概括转让（第五百五十四条）
1. 需经对方同意
2. 适用规则（第五百五十五条）：同时适用债权转让和债务转让的规定

注：第五章"占有"及后续关于地役权、担保物权等内容与合同变更转让无直接关联，故未纳入本回答。


In [102]:
question="宅基地使用权"

In [103]:
search_res = milvus_client.search(
    collection_name=collection_name,
    data=embedding_model.encode_queries([question]),
    limit=5,
    search_params={ "metric_type": "IP", "params": {} },
    output_fields=["text"]
)

In [104]:
retrieved_lines_with_distances = [
    (res["entity"]["text"], res["distance"]) for res in search_res[0]
]

In [105]:
context = "\n".join(
    (line_with_distance[0]) for line_with_distance in retrieved_lines_with_distances
)

In [106]:
SYSTEM_PROMPT = """
你是一个 ai 问答小助手，你可以使用提供的上下文回答问题
"""
USER_PROMPT = f"""
请使用<context>标签内的上下文来回答<question>标签内的问题
<question>
{question}
</question>
<context>
{context}
</context>
"""

In [ ]:
response=deepseek_client.chat.completions.create(
    model="deepseek-chat",
    messages=[
        {"role": "system", "content": SYSTEM_PROMPT},
        {"role": "user", "content": USER_PROMPT}
    ]
)
print(response.choices[0].message.content)

根据提供的上下文，关于**宅基地使用权**的明确规定主要出现在**地役权**一节（第三百九十五条），具体内容如下：

1. **宅基地使用权的转让与地役权的关系**  
   第三百九十五条规定：  
   - 地役权不得单独转让或抵押。但若**宅基地使用权**等权利发生转让或抵押时，地役权需一并转让或抵押。  
   - 即宅基地使用权的处分（如转让、抵押）将导致其附属的地役权同步转移。

2. **宅基地使用权的法律属性**  
   上下文虽未直接定义宅基地使用权，但结合相关条款可推断其属于用益物权的一种：  
   - 类似**土地承包经营权**（第三百五十八条），宅基地使用权人应享有对集体土地的占有、使用和收益权。  
   - 其转让需遵循物权登记规则（如第三百九十条），未经登记不得对抗善意第三人。

3. **其他关联规则**  
   - 若宅基地涉及地役权（如通行、排水等），需遵守地役权合同的约定（第三百八十九条）。  
   - 担保物权实现时（如抵押宅基地），担保权人可优先受偿（第三百九十九条）。

**结论**：上下文未系统规定宅基地使用权的全部内容，但明确其转让或抵押时需连带处理地役权，并受物权登记、合同约定等一般规则约束。具体权能可参考土地承包经营权的类似条款（占有、使用、收益）。
